 ## Task 4：源大模型微调实战

### 2.1 环境准备

In [1]:
# 查看已安装依赖
! pip list

Package                        Version
------------------------------ --------------
absl-py                        2.1.0
accelerate                     0.33.0
aiohappyeyeballs               2.4.0
aiohttp                        3.10.5
aiosignal                      1.3.1
altair                         5.4.0
anyio                          4.4.0
argon2-cffi                    23.1.0
argon2-cffi-bindings           21.2.0
arrow                          1.3.0
asttokens                      2.4.1
async-lru                      2.0.4
async-timeout                  4.0.3
attrs                          23.2.0
Babel                          2.15.0
beautifulsoup4                 4.12.3
bleach                         6.1.0
blinker                        1.8.2
brotlipy                       0.7.0
cachetools                     5.5.0
certifi                        2022.12.7
cffi                           1.15.1
charset-normalizer             2.0.4
click                          8.1.7
comm           

In [2]:
# 安装 streamlit
! pip install streamlit==1.24.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
# 安装 modelscope 库
! pip install modelscope

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [4]:
# 安装 datasets 、transformers库
! pip install datasets
! pip install transformers


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [5]:
# 安装、sentencepiece 、einops 、peft库
! pip install sentencepiece
! pip install einops
! pip install peft
! pip install -U accelerate

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [6]:
# 更新 accelerate库
! pip install -U accelerate

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


### 2.2 模型下载

In [7]:
from modelscope import snapshot_download
model_dir = snapshot_download('IEITYuan/Yuan2-2B-Mars-hf', cache_dir='.')

### 2.3 数据处理

In [8]:
# 导入环境
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

In [9]:
# 读取数据
df = pd.read_json('./data.json')
ds = Dataset.from_pandas(df)

In [10]:
# 查看数据
len(ds)
ds[:1]

{'input': ['# 任务描述\n假设你是一个AI简历助手，能从简历中识别出所有的命名实体，并以json格式返回结果。\n\n# 任务要求\n实体的类别包括：姓名、国籍、种族、职位、教育背景、专业、组织名、地名。\n返回的json格式是一个字典，其中每个键是实体的类别，值是一个列表，包含实体的文本。\n\n# 样例\n输入：\n张三，男，中国籍，工程师\n输出：\n{"姓名": ["张三"], "国籍": ["中国"], "职位": ["工程师"]}\n\n# 当前简历\n高勇：男，中国国籍，无境外居留权，\n\n# 任务重述\n请参考样例，按照任务要求，识别出当前简历中所有的命名实体，并以json格式返回结果。'],
 'output': ['{"姓名": ["高勇"], "国籍": ["中国国籍"]}']}

In [11]:
# 加载 tokenizer
path = './IEITYuan/Yuan2-2B-Mars-hf'

# 使用默认的 legacy 行为
tokenizer = AutoTokenizer.from_pretrained(path, add_eos_token=False, add_bos_token=False, eos_token='<eod>', legacy=True)

# 添加额外的特殊标记
tokenizer.add_tokens(['<sep>', '<pad>', '<mask>', '<predict>', '<FIM_SUFFIX>', '<FIM_PREFIX>', '<FIM_MIDDLE>', '<commit_before>', '<commit_msg>', '<commit_after>', '<jupyter_start>', '<jupyter_text>', '<jupyter_code>', '<jupyter_output>', '<empty_output>'], special_tokens=True)
tokenizer.pad_token = tokenizer.eos_token

# 如果您希望使用新行为
tokenizer_new = AutoTokenizer.from_pretrained(path, add_eos_token=False, add_bos_token=False, eos_token='<eod>', legacy=False)
tokenizer_new.add_tokens(['<sep>', '<pad>', '<mask>', '<predict>', '<FIM_SUFFIX>', '<FIM_PREFIX>', '<FIM_MIDDLE>', '<commit_before>', '<commit_msg>', '<commit_after>', '<jupyter_start>', '<jupyter_text>', '<jupyter_code>', '<jupyter_output>', '<empty_output>'], special_tokens=True)
tokenizer_new.pad_token = tokenizer_new.eos_token

In [12]:
# 定义数据处理函数
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性

    instruction = tokenizer(f"{example['input']}<sep>")
    response = tokenizer(f"{example['output']}<eod>")
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = [1] * len(input_ids) 
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] # instruction 不计算loss

    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [13]:
# 处理数据集
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 200
})

In [14]:
# 数据检查
tokenizer.decode(tokenized_id[0]['input_ids'])

'# 任务描述\n假设你是一个AI简历助手，能从简历中识别出所有的命名实体，并以json格式返回结果。\n\n# 任务要求\n实体的类别包括：姓名、国籍、种族、职位、教育背景、专业、组织名、地名。\n返回的json格式是一个字典，其中每个键是实体的类别，值是一个列表，包含实体的文本。\n\n# 样例\n输入：\n张三，男，中国籍，工程师\n输出：\n{"姓名": ["张三"], "国籍": ["中国"], "职位": ["工程师"]}\n\n# 当前简历\n高勇：男，中国国籍，无境外居留权，\n\n# 任务重述\n请参考样例，按照任务要求，识别出当前简历中所有的命名实体，并以json格式返回结果。<sep> {"姓名": ["高勇"], "国籍": ["中国国籍"]}<eod>'

In [15]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"])))

'{"姓名": ["高勇"], "国籍": ["中国国籍"]}<eod>'

### 2.4 模型训练

移除模型加载时的device_map="auto" 和 torch_dtype=torch.bfloat16 参数，因为这些设置默认会将模型加载到GPU上（如果可用）并使用bfloat16精度来节省内存。在CPU上，我们不需要指定设备映射，也不需要使用bfloat16，因为它主要用于加速GPU计算。

In [16]:
# 模型加载
#model = AutoModelForCausalLM.from_pretrained(path, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
#model

device = torch.device("cpu")
model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True)
model.to(device)  # 显式地将模型移动到CPU上

/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


YuanForCausalLM(
  (model): YuanModel(
    (embed_tokens): Embedding(135040, 2048, padding_idx=77185)
    (layers): ModuleList(
      (0-23): 24 x YuanDecoderLayer(
        (self_attn): YuanAttention(
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): YuanRotaryEmbedding()
          (lf_gate): LocalizedFiltering(
            (conv1): Conv2d(2048, 1024, kernel_size=(2, 1), stride=(1, 1), padding=(1, 0))
            (conv2): Conv2d(1024, 2048, kernel_size=(2, 1), stride=(1, 1), padding=(1, 0))
            (output_layernorm): YuanRMSNorm()
          )
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): YuanMLP(
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (gate_proj): Linear(in_features=2048, out_featu

In [17]:
model.enable_input_require_grads() # 开启gradient_checkpointing时，要执行该方法

In [18]:
# 查看模型数据类型
model.dtype

torch.float32

In [19]:
# 配置Lora
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'up_proj', 'down_proj', 'q_proj', 'gate_proj', 'o_proj', 'k_proj', 'v_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [20]:
# 构建PeftModel
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): YuanForCausalLM(
      (model): YuanModel(
        (embed_tokens): Embedding(135040, 2048, padding_idx=77185)
        (layers): ModuleList(
          (0-23): 24 x YuanDecoderLayer(
            (self_attn): YuanAttention(
              (v_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (o_proj):

In [21]:
# 打印需要训练的参数
model.print_trainable_parameters()

trainable params: 9,043,968 || all params: 2,097,768,448 || trainable%: 0.4311


建议你在使用gradient_checkpointing时显式指定use_reentrant参数
让我们先解决use_reentrant的警告。你可以在TrainingArguments中添加use_reentrant=False来解决这个问题。这是修改后的部分代码：

In [22]:
# 设置训练参数
args = TrainingArguments(
    output_dir="./output/Yuan2.0-2B_lora_bf16",
    per_device_train_batch_size=12,
    gradient_accumulation_steps=1,
    logging_steps=1,
    save_strategy="epoch",
    num_train_epochs=3,
    learning_rate=5e-5,
    save_on_each_node=True,
    gradient_checkpointing=True,
    bf16=False,  # 关闭bf16，因为我们正在使用CPU
    fp16=False,  # 同样，关闭fp16
    # 注意这里不再包含use_reentrant
)

In [23]:
# 初始化Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [ ]:
# 模型训练
trainer.train()

/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.290400
2,1.095600
3,0.681400
4,0.575100
5,0.500400
6,0.541800
7,0.427000
8,0.627300
9,0.398900
10,0.301000


/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


### 2.5 效果验证

调整generate函数中的.cuda()调用。.cuda()会将张量移动到GPU上。由于我们要在CPU上运行，你需要移除这个调用，或者将其替换为.to(device)，其中device被设置为torch.device("cpu")。

In [ ]:
# 定义生成函数
def generate(prompt):
    prompt = prompt + "<sep>"
    #inputs = tokenizer(prompt, return_tensors="pt")["input_ids"].cuda()
    inputs = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)
    outputs = model.generate(inputs, do_sample=False, max_length=256)
    output = tokenizer.decode(outputs[0])
    print(output.split("<sep>")[-1])

In [ ]:
# 输入prompt template
template = '''
# 角色
你是一个资深的 AI 翻译助理，具备卓越的语言能力，能够精准、流畅且地道地翻译英语文本。

# 技能
1. 当用户提供英语，你输出中文

# 回复示例
输入： Previously we obtained recombinant soluble human rhIFN-λ1 from Pichia pastoris.
输出: 此前，我们已经从毕赤酵母表达获得了可溶性重组人干扰素-λ1。

# 限制:
- 仅专注于翻译相关的任务，坚决拒绝回答与翻译无关的任何问题。
- 译文要清晰易懂、自然流畅，避免使用生僻或过于复杂的中文表述。

input_str

'''


In [ ]:
input_str = '"Results: Fourteen patients, including 5 males and 9 females, were (55±8) years old, there were 9 patients with single lesion and 5 patients with multiple lesions (2-6 lesions). a total of 26 lesions were identified."'
prompt = template.replace('input_str', input_str).strip()
generate(prompt)